# Cours LangChain TP2

Dans ce TP vous allez voir comment créer des chaines simples et comment utilser des Agents

install et import de clé API

In [ ]:
!pip install -U langchain-openai langchain docarray tiktoken

In [3]:
import os
os.environ['OPENAI_API_KEY']="OPENAI_API_KEY" # À Modifier

## Tâche 1 : Créer une chaîne simple

* Objectif : Les élèves créeront une chaîne simple impliquant une séquence d'appels à un modèle de langage ou à d'autres utilitaires.
* Partie pratique : Les élèves définissent une séquence d'appels, intègrent des outils et exécutent la chaîne pour observer la séquence des actions.

### Exercice 1 
Il y a un document texte racontant une histoire fictive d'un detective privé
Le but de ce TP est de créer des chaines simples à partir de modules vus en cours pour les combiner ensembles

Tout d'abord chargez et splitter le document texte `document.txt`, puis mettez le en vectorb db locale avec `DocArrayInMemorySearch`

In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

#TO DO
documents = TextLoader("document.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
docs

[Document(page_content="Aaron, âgé de 35 ans, vit dans la ville animée de New York, où il est détective privé.\nNé dans une famille nombreuse, Aaron est le deuxième de quatre enfants. \nIl a deux frères, James et Marcus, ainsi qu'une sœur défunte, Emily, décédée tragiquement dans un accident de voiture il y a quelques années.", metadata={'source': 'document.txt'}),
 Document(page_content='Depuis son plus jeune âge, Aaron a été fasciné par les mystères et les énigmes. \nAprès avoir obtenu son diplôme en criminologie, il a décidé de suivre les traces de son père, un ancien policier, et de devenir détective privé.', metadata={'source': 'document.txt'}),
 Document(page_content="Aaron mène une vie solitaire et dévouée à son travail. \nIl passe ses journées à résoudre des affaires complexes, arpentant les rues de la ville à la recherche d'indices et de témoins. \nSon bureau est un sanctuaire de dossiers et de documents, où il passe des heures à analyser les détails les plus infimes pour réso

### Exercice 2 
Utiliser cette base de donnée en tant que retriever 

In [6]:
# TO DO
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever()

### Exercice 3 

Créer un prompt dans lequel vous allez rajouter une variable `context` qui sera le contexte du retriever et une variable `question` pour demander à l'utilisateurs des questions sur l'histoire du document texte

In [9]:
from langchain_core.prompts import PromptTemplate

#TO DO
template = """Réponds aux questions en te servant du contexte suivant:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['context', 'question'], template='Réponds aux questions en te servant du contexte suivant:\n{context}\n\nQuestion: {question}\n')

### Exercice 4
Utiliser les réponse précedentes pour créer une chaines qui utilisera le retriever pour répondre au questions de l'utilisateur

In [10]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAI
from langchain_core.output_parsers import StrOutputParser


#utiliser le retriever en element à passer à la chaine 
# TO DO
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

model = OpenAI()
output_parser = StrOutputParser()

#declarer un OutputParser pour sortir le contenu en chain de charactères simples

#declarer la chaine
chain = setup_and_retrieval | prompt | model | output_parser #TO DO

In [13]:
from langchain.callbacks import StdOutCallbackHandler


chain.invoke("Quels étaient les frères et soeurs du personnage principal?")

'Réponse: Les frères du personnage principal sont James et Marcus, et sa sœur défunte est Emily.'

In [21]:
#Solution
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough


documents = TextLoader("document.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

db = FAISS.from_documents(docs, embeddings)

retriever = db.as_retriever()

template = """Réponds aux questions en te servant du contexte suivant:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)
model = OpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("Quels étaient les frères et soeurs du personnage principal?",config = {'callbacks' : [handler]})



> Entering new RunnableSequence chain...


> Entering new RunnableParallel<context,question> chain...


> Entering new RunnablePassthrough chain...

> Finished chain.

> Finished chain.


> Entering new PromptTemplate chain...

> Finished chain.


> Entering new StrOutputParser chain...

> Finished chain.

> Finished chain.


'Réponse: Les frères et sœur du personnage principal sont James, Marcus et Emily.'

## Tâche 2 : Mise en œuvre d'un agent

* Objectif : Les élèves mettront en œuvre un agent qui utilise un LLM comme moteur de raisonnement pour prendre des décisions sur les actions à entreprendre.
* Partie pratique : Les élèves définiront un agent, créeront des outils et utiliseront l'`AgentExecutor` pour exécuter l'agent et observer le processus de prise de décision basé sur les réponses du modèle de langage.

Le but sera de créer un agent qui va récuperer ce document et de continuer l'histoire. À partir des réponse précedentes et de vos recherches sur la documentation Langchain
Créez deux tools appelant le retriever:
* Un qui aura comme instruction de continuer l'histoire
* Un qui devra juste répondre aux question de l'utilisateurs

Créer un agent qui intègre ces deux outils et call un LLM

In [ ]:
#Solution
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, create_openai_tools_agent
#Load document
#TODO

#retriever 
#TODO

#Creation des deux tools
#TODO

#Prompt
#TODO

#Création d'un Agent
#TODO

In [36]:
#Solution
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, create_openai_tools_agent
#Load document
documents = TextLoader("document.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)

#retriever 
retriever = db.as_retriever()

#Creation des deux tools

tool_a = create_retriever_tool(
    retriever,
    "continue_story",
    "Continue l'histoire avec un nouveau crime",
)
tool_b = create_retriever_tool(
    retriever,
    "explain_story",
    "Réponds à la question de l'utilisateurs sur l'histoire",
)
tools = [tool_a,tool_b]

#Prompt from hub
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages[0].prompt.template="Tu es un assistant IA serviable." # Force le prompt à être en fr

#Création d'un Agent
llm = ChatOpenAI(temperature=0)
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True)


In [37]:
agent_executor.invoke({"input": "Quels étaient les frères d'Aaron?"})



> Entering new AgentExecutor chain...

Invoking: `explain_story` with `{'query': "frères d'Aaron"}`


Malgré son emploi du temps chargé, Aaron reste proche de ses frères, James et Marcus, qui le soutiennent dans son travail difficile. 
Ils se réunissent souvent pour dîner en famille et se remémorer les souvenirs de leur enfance.

Aaron, âgé de 35 ans, vit dans la ville animée de New York, où il est détective privé.
Né dans une famille nombreuse, Aaron est le deuxième de quatre enfants. 
Il a deux frères, James et Marcus, ainsi qu'une sœur défunte, Emily, décédée tragiquement dans un accident de voiture il y a quelques années.

La perte de leur sœur Emily a laissé une cicatrice indélébile dans la vie d'Aaron, et il garde toujours une photo d'elle sur son bureau en guise de rappel de sa mémoire.
Sa mort l'a motivé à rechercher la justice pour les autres et à mettre les criminels derrière les barreaux, en espérant ainsi éviter à d'autres familles la douleur qu'ils ont ressentie.

Depuis

{'input': "Quels étaient les frères d'Aaron?",
 'output': 'Aaron a deux frères, James et Marcus. Ils sont proches et se soutiennent mutuellement malgré les défis de la vie.'}

In [38]:
agent_executor.invoke({"input": "Continue l'histoire du détective."})



> Entering new AgentExecutor chain...

Invoking: `continue_story` with `{'query': 'continue story'}`


Malgré son emploi du temps chargé, Aaron reste proche de ses frères, James et Marcus, qui le soutiennent dans son travail difficile. 
Ils se réunissent souvent pour dîner en famille et se remémorer les souvenirs de leur enfance.

Malgré les défis et les dangers de son métier, Aaron reste déterminé à faire tout ce qui est en son pouvoir pour rendre la ville plus sûre et plus juste pour tous ses habitants.

La perte de leur sœur Emily a laissé une cicatrice indélébile dans la vie d'Aaron, et il garde toujours une photo d'elle sur son bureau en guise de rappel de sa mémoire.
Sa mort l'a motivé à rechercher la justice pour les autres et à mettre les criminels derrière les barreaux, en espérant ainsi éviter à d'autres familles la douleur qu'ils ont ressentie.

Depuis son plus jeune âge, Aaron a été fasciné par les mystères et les énigmes. 
Après avoir obtenu son diplôme en criminologie, 

{'input': "Continue l'histoire du détective.",
 'output': "L'histoire du détective Aaron se poursuit avec son engagement à rendre la ville plus sûre et plus juste, tout en restant proche de sa famille malgré les défis de son métier. Sa détermination à rechercher la justice pour les autres est motivée par la perte de sa sœur Emily. Son parcours en tant que détective privé est marqué par sa passion pour les mystères et les énigmes, héritée de son père policier."}